In [1]:
!nvidia-smi

zsh:1: command not found: nvidia-smi


In [2]:
!lscpu

Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          48 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   12
  On-line CPU(s) list:    0-11
Vendor ID:                AuthenticAMD
  Model name:             AMD Ryzen 5 5600X 6-Core Processor
    CPU family:           25
    Model:                33
    Thread(s) per core:   2
    Core(s) per socket:   6
    Socket(s):            1
    Stepping:             2
    Frequency boost:      enabled
    CPU(s) scaling MHz:   95%
    CPU max MHz:          4651.0000
    CPU min MHz:          550.0000
    BogoMIPS:             7402.24
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge m
                          ca cmov pat pse36 clflush mmx fxsr sse sse2 ht syscall
                           nx mmxext fxsr_opt pdpe1gb rdtscp lm constant_tsc rep
                          _good nopl xtopology nonstop_tsc cpuid extd_apicid ape
   

In [5]:
!pip install pandas torch torchvision transformers pytorch-lightning tqdm ipywidgets

  Obtaining dependency information for ipywidgets from https://files.pythonhosted.org/packages/22/2d/9c0b76f2f9cc0ebede1b9371b6f317243028ed60b90705863d493bae622e/ipywidgets-8.1.5-py3-none-any.whl.metadata
  Obtaining dependency information for widgetsnbextension~=4.0.12 from https://files.pythonhosted.org/packages/21/02/88b65cc394961a60c43c70517066b6b679738caf78506a5da7b88ffcb643/widgetsnbextension-4.0.13-py3-none-any.whl.metadata
  Obtaining dependency information for jupyterlab-widgets~=3.0.12 from https://files.pythonhosted.org/packages/a9/93/858e87edc634d628e5d752ba944c2833133a28fa87bb093e6832ced36a3e/jupyterlab_widgets-3.0.13-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 2.2 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 5.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 16.9 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] 

In [7]:
import pandas as pd
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import AutoProcessor, AutoModelForZeroShotImageClassification
from tqdm import tqdm

In [5]:
!cp /kaggle/input/goldenleaf/image_descriptions.csv .

/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [6]:
!mkdir -p src/resources
!cp -r /kaggle/input/goldenleaf/images src/resources

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Load model directly

processor = AutoProcessor.from_pretrained("openai/clip-vit-large-patch14")
model = AutoModelForZeroShotImageClassification.from_pretrained("openai/clip-vit-large-patch14")

In [14]:
data = pd.read_csv("image_descriptions.csv")


class ImageTextDataset(Dataset):
    def __init__(self, data, processor):
        self.data = data
        self.processor = processor
        self.images = self.load_images()

    def load_images(self):
            loaded_images = {}
            for idx in tqdm(range(len(self.data)), desc="Loading Images"):
                image_path = self.data.iloc[idx]["image"]
                image = Image.open(image_path).convert("RGB")
                image = image.resize((224, 224))
                loaded_images[image_path] = image
            return loaded_images

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_path = self.data.iloc[idx]["image"]
        description = self.data.iloc[idx]["description"]
        image = self.images[image_path]

        return {
            "image": image,
            "text": description
        }


In [15]:
dataset = ImageTextDataset(data, processor)

train_size = int(0.8 * len(data))
val_size = len(data) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

Loading Images: 100%|██████████| 115/115 [00:26<00:00,  4.42it/s]


In [17]:
def collate_fn(batch):
    texts = [item['text'] for item in batch]
    images = [item['image'] for item in batch]

    # Utilisation du processor pour traiter les données
    inputs = processor(
        text=texts,
        images=images,
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=77
    )

    return inputs


batch_size = 4
train_dataloader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,  # Shuffle pour l'entraînement
    collate_fn=collate_fn,
    num_workers=2,
)

val_dataloader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,  # Pas besoin de shuffle pour la validation
    collate_fn=collate_fn,
    num_workers=2,
)


In [29]:
import torch
import torch.nn as nn
import torch.optim as optim
import pytorch_lightning as pl

class CLIPFinetuner(pl.LightningModule):
    def __init__(self, model, lr=1e-5):
        super().__init__()
        self.model = model
        self.lr = lr
        self.loss_fn = nn.CrossEntropyLoss()

    def forward(self, pixel_values, input_ids, attention_mask):
        return self.model(
            pixel_values=pixel_values,
            input_ids=input_ids,
            attention_mask=attention_mask,
        )

    def compute_loss(self, logits_per_image, logits_per_text):
        labels = torch.arange(logits_per_image.size(0), device=self.device)
        loss_img_to_txt = self.loss_fn(logits_per_image, labels)
        loss_txt_to_img = self.loss_fn(logits_per_text, labels)
        return (loss_img_to_txt + loss_txt_to_img) / 2

    def training_step(self, batch, batch_idx):
        outputs = self(**batch)
        loss = self.compute_loss(
            logits_per_image=outputs.logits_per_image,
            logits_per_text=outputs.logits_per_text,
        )
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        outputs = self(**batch)
        loss = self.compute_loss(
            logits_per_image=outputs.logits_per_image,
            logits_per_text=outputs.logits_per_text,
        )
        self.log("val_loss", loss, on_epoch=True, prog_bar=True)

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=self.lr)

        # Define a learning rate scheduler
        scheduler = {
            'scheduler': optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1),
            'monitor': 'val_loss',
            'interval': 'epoch',  # Adjust every epoch
            'frequency': 1,  # Apply scheduler every epoch
        }

        return [optimizer], [scheduler]



In [32]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import CSVLogger

# Initialisation du modèle Lightning
model = CLIPFinetuner(model)

# Configuration des callbacks (sauvegarde du meilleur modèle)
checkpoint_callback = ModelCheckpoint(
    monitor="val_loss", mode="min", save_top_k=1, filename="clip_finetuned-{epoch:02d}-{val_loss:.4f}"
)

# Configuration du logger pour suivre les métriques
logger = CSVLogger("logs", name="clip_finetuning")

# Initialisation du trainer
trainer = Trainer(
    max_epochs=10,
    accelerator="cpu",  # Utilise le GPU si disponible
    devices=1,          # Nombre de GPU
    callbacks=[checkpoint_callback],
    logger=logger,
    log_every_n_steps=8,
    enable_progress_bar=True,
    accumulate_grad_batches=4,
)

# Entraînement
trainer.fit(model, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name    | Type             | Params | Mode 
-----------------------------------------------------
0 | model   | CLIPFinetuner    | 427 M  | train
1 | loss_fn | CrossEntropyLoss | 0      | train
-----------------------------------------------------
427 M     Trainable params
0         Non-trainable params
427 M     Total params
1,710.466 Total estimated model params size (MB)
5         Modules in train mode
462       Modules in eval mode


Epoch 0: 100%|██████████| 23/23 [02:04<00:00,  0.19it/s, v_num=5, train_loss_step=1.390]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 1: 100%|██████████| 23/23 [02:07<00:00,  0.18it/s, v_num=5, train_loss_step=1.320, val_loss=1.380, train_loss_epoch=1.390]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 2: 100%|██████████| 23/23 [02:03<00:00,  0.19it/s, v_num=5, train_loss_step=1.360, val_loss=1.460, train_loss_epoch=1.350]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 3: 100%|██████████| 23/23 [02:02<00:00,  0.19it/s, v_num=5, train_loss_step=0.880, val_loss=2.210, train_loss_epoch=1.260]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 4: 100%|██████████| 23/23 [02:04<00:00,  0.18it/s, v_num=5, train_loss_step=1.010, val_loss=2.340, train_loss_epoch=0.990] 
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 5: 100%|██████████| 23/23 [01:55<00:00,  0.20it/s, v_num=5, train_loss_step=0.174, val_loss=2.650, train_loss_epoch=0.647]
Validation: |          | 0/? [00:00

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 23/23 [02:10<00:00,  0.18it/s, v_num=5, train_loss_step=0.419, val_loss=2.880, train_loss_epoch=0.338]


In [34]:
from pytorch_lightning import Trainer

# Load model from checkpoint
checkpoint_path = "logs/clip_finetuning/version_5/checkpoints/clip_finetuned-epoch=00-val_loss=1.3814.ckpt"
model = CLIPFinetuner.load_from_checkpoint(checkpoint_path, model=model)

RuntimeError: Error(s) in loading state_dict for CLIPFinetuner:
	Missing key(s) in state_dict: "model.model.model.model.model.model.model.model.logit_scale", "model.model.model.model.model.model.model.model.text_model.embeddings.token_embedding.weight", "model.model.model.model.model.model.model.model.text_model.embeddings.position_embedding.weight", "model.model.model.model.model.model.model.model.text_model.encoder.layers.0.self_attn.k_proj.weight", "model.model.model.model.model.model.model.model.text_model.encoder.layers.0.self_attn.k_proj.bias", "model.model.model.model.model.model.model.model.text_model.encoder.layers.0.self_attn.v_proj.weight", "model.model.model.model.model.model.model.model.text_model.encoder.layers.0.self_attn.v_proj.bias", "model.model.model.model.model.model.model.model.text_model.encoder.layers.0.self_attn.q_proj.weight", "model.model.model.model.model.model.model.model.text_model.encoder.layers.0.self_attn.q_proj.bias", "model.model.model.model.model.mode

In [33]:
torch.save(model.state_dict(), "clip_finetuned.pth")